In [2]:
from deap import base, creator, tools
import random
import numpy as np
from sklearn.linear_model import LinearRegression, LassoLarsIC, Lasso
import pandas as pd

In [3]:
X = pd.read_csv('a1_data.csv') #names=['A','B','C','D','E','F','G','H','I','J','K']
X = X.values
X

array([[3.17966504, 3.03062471, 3.22275227, ..., 3.70514307, 3.92179743,
        3.5667659 ],
       [3.31315547, 3.12753458, 3.36718872, ..., 3.00232446, 3.48703507,
        3.75358101],
       [3.5318474 , 3.55448902, 3.44163644, ..., 3.53159902, 3.77334177,
        3.25474063],
       ...,
       [3.55076635, 3.48014853, 3.65358394, ..., 2.69928466, 0.9630304 ,
        3.1252519 ],
       [3.64113611, 3.67916051, 3.13758092, ..., 4.34764293, 3.85532063,
        3.21161165],
       [3.58635986, 2.83222446, 3.24274907, ..., 2.69928466, 0.73012916,
        3.04170596]])

In [4]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [5]:
IND_SIZE = 10

# TODO SEED

toolbox = base.Toolbox()

def sortnregress(X, alpha=0.01):
    lasso = Lasso(alpha)

    d = X.shape[1]
    W = np.zeros((d, d))
    increasing = np.argsort(np.var(X, axis=0))
    for k in range(1, d):
        covariates = increasing[:k]
        target = increasing[k]
        lasso.fit(X[:, covariates], X[:, target].ravel())
        weight = np.abs(lasso.coef_)
        W[covariates, target] = weight
    return W

def create_individual():
    # TODO better alpha 
    return sortnregress(X, alpha=random.random()/10)

# creating population
toolbox.register("attribute", create_individual)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [6]:
# creating fitness function
def evaluate(individual):
    return sum(individual),

def mate(ind1, ind2):
    ind1 = np.array(ind1)
    ind2 = np.array(ind2)

    child1 = np.zeros(ind1.shape)
    child2 = np.zeros(ind1.shape)

    for (i, j), _ in np.ndenumerate(ind1):
        # assuming as if 1/0 graph
        # TODO check for values close to 0
        if ind1[i,j] == 0 and ind2[i,j] == 0:
            child1[i,j] = 0
            child2[i,j] = 0
        elif ind1[i,j] != 0 and ind2[i,j] != 0:
            child1[i,j] = 1
            child2[i,j] = 1
        else:
            child1[i,j] = random.randint()
            child2[i,j] = random.randint()

    # TODO regression for values

    return child1, child2

def mutate(ind):
    # create list of indices of non-zero elements
    ind = np.array(ind)
    ind = ind.ravel()
    non_zero = np.nonzero(ind)[0]
    
    # choose random index
    index = random.choice(non_zero)

    # mutate
    ind[index] = 0

    # TODO more complex mutation, adding edges?

    return ind,

def evaluate(ind):
    # make sure the individual still fulfils the requirements of a DAG
    # TODO

    return ind,

toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [7]:
# TODO this is just copied from the docs:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop